In [594]:
import pandas as pd
import os
import json
import glob
import re
import typing
import math
from functools import lru_cache

pd.options.display.max_rows = 100

fields = pd.read_json('raw/fields-2012_2025.json')
standards = pd.read_json('raw/standards-2020_2025.json')
downhill_marathons = pd.read_csv('raw/downhill-marathons.csv')

## Merging Downhill Marathon Results

In [574]:
athlinks_slugs = ['tucson', 'sundance-to-spearfish', 'pocatello', 'leading-ladies', 'jack-and-kills', 'hawaii-bird-conservation']
raceentry_slugs = ['utah-valley']
ultrasignup_slugs = ['tunnel-vision', 'tunnel-light', 'light-at-the-end-of-the-tunnel']
chronokeep_slugs = ['super', 'cascade-express']
webscorer_slugs = ['sun', 'east-canyon']
runsignup_slugs = ['runtastic-nebo']
brooksee_slugs = ['revel-white-mountains', 'revel-mt-charleston', 'revel-big-bear', 'revel-big-cottonwood', 'deseret-news', 'bears-ears']

race_slugs = {
    'athlinks': athlinks_slugs,
    'raceentry': raceentry_slugs,
    'ultrasignup': ultrasignup_slugs,
    'chronokeep': chronokeep_slugs,
    'webscorer': webscorer_slugs,
    'runsignup': runsignup_slugs,
    'brooksee': brooksee_slugs
}

In [575]:
def conv_formatted_time_to_millis(formatted_time: str):
    """Convert a formatted time string to milliseconds."""
    if formatted_time in ['DNF', '---', '–'] or pd.isna(formatted_time):
        return None
    parts = formatted_time.split(':')
    if len(parts) == 3:  # HH:MM:SS
        hours, minutes, seconds = map(float, parts)
        return (hours * 3600 + minutes * 60 + seconds) * 1000
    elif len(parts) == 2:  # MM:SS
        minutes, seconds = map(float, parts)
        return (minutes * 60 + seconds) * 1000
    else:
        raise ValueError(f'Invalid time format: {formatted_time}')

In [576]:
def split_name_affiliation(name_affiliation: str):
    parts = re.split(r', ([a-z0-9A-Z][A-Z])', name_affiliation)
    if len(parts) < 3:
        return [name_affiliation.strip(), '']
    middle_part = parts[1]
    return [parts[0].strip() + middle_part[0], middle_part[1] + parts[2].strip()]

In [577]:
downhill_marathon_results_df = pd.DataFrame(columns=['slug', 'date', 'name', 'time', 'gender', 'age', 'city', 'state'])

for provider in race_slugs:
    for race_slug in race_slugs[provider]:
        file_paths = glob.glob(f'raw/marathons/{race_slug}-*.{"json" if provider == "athlinks" else "csv"}')

        for file_path in file_paths:
            date = '-'.join(os.path.basename(file_path).split('-')[-3:]).split('.')[0]
            if provider == 'athlinks':
                with open(file_path, 'r') as file:
                    data = json.load(file)
                results = data['intervals'][0]['results']
                temp_df = pd.DataFrame(results)
                temp_df['locality'] = temp_df['location'].apply(lambda x: x.get('locality', None) if x is not None else None)
                temp_df['region'] = temp_df['location'].apply(lambda x: x.get('region', None) if x is not None else None)
                temp_df = temp_df[['age', 'gender', 'chipTimeInMillis', 'displayName', 'locality', 'region']]
            elif provider == 'raceentry' or provider == 'runsignup' or provider == 'brooksee':
                temp_df = pd.read_csv(file_path)
                if provider == 'raceentry':
                    temp_df['name'] = temp_df['First Name'] + ' ' + temp_df['Last Name']
                    temp_df['city'] = temp_df['City']
                    temp_df['state'] = temp_df['State']
                elif provider == 'runsignup':
                    temp_df['name'] = temp_df['Name'].apply(lambda x: ' '.join(x.split()[1:]))
                    temp_df['city'] = temp_df['City']
                    temp_df['state'] = temp_df['State']
                elif provider == 'brooksee':
                    temp_df['name'] = temp_df['First Name'] + ' ' + temp_df['Last Name']
                    temp_df['city'] = None
                    temp_df['state'] = None
                temp_df = temp_df[['Age', 'Gender', 'Chip Time', 'name', 'city', 'state']]
                temp_df['Chip Time'] = temp_df['Chip Time'].apply(conv_formatted_time_to_millis)
            elif provider == 'ultrasignup' or (race_slug == 'super' and date.startswith('2022')) or (race_slug == 'cascade-express' and date.startswith('2023')):
                temp_df = pd.read_csv(file_path)
                temp_df['name'] = temp_df['first_name'] + ' ' + temp_df['last_name']
                temp_df = temp_df[['age', 'gender', 'time_millis', 'name', 'city', 'state']]
            elif provider == 'chronokeep':
                temp_df = pd.read_csv(file_path)
                temp_df['city'] = None
                temp_df['state'] = None
                temp_df = temp_df[['Age', 'Gender', 'Chip Time*', 'Name', 'city', 'state']]
                temp_df['Chip Time*'] = temp_df['Chip Time*'].apply(conv_formatted_time_to_millis)
            elif provider == 'webscorer':
                temp_df = pd.read_csv(file_path)
                name_affiliations = temp_df['Name Affiliation'].apply(split_name_affiliation)
                temp_df['name'] = name_affiliations.apply(lambda x: x[0])
                temp_df['city'] = name_affiliations.apply(lambda x: x[1].split(',')[0].strip() if ',' in x[1] else None)
                temp_df['state'] = name_affiliations.apply(lambda x: x[1].split(',')[1].strip() if ',' in x[1] else None)
                temp_df = temp_df[['Age', 'Gender', 'Finish time 			Chip time', 'name', 'city', 'state']]
                temp_df['Finish time 			Chip time'] = temp_df['Finish time 			Chip time'].apply(conv_formatted_time_to_millis)
            else:
                continue

            temp_df.columns = ['age', 'gender', 'time', 'name', 'city', 'state']
            temp_df['slug'] = race_slug
            temp_df['date'] = pd.Timestamp(date)
            downhill_marathon_results_df = pd.concat([downhill_marathon_results_df, temp_df])

downhill_marathon_results_df

,slug,date,name,time,gender,age,city,state
0,tucson,2022-12-10,Rob Arend,9420000,M,29.0,Phoenix,Arizona
1,tucson,2022-12-10,Dan Brownstein,9461000,M,26.0,Ticonderoga,New York
2,tucson,2022-12-10,Nick Coury,9513000,M,35.0,Scottsdale,Arizona
3,tucson,2022-12-10,Michael Tomchaney,9647000,M,33.0,Oro Valley,Arizona
4,tucson,2022-12-10,Drew Frehs,9661000,M,35.0,Tucson,Arizona
...,...,...,...,...,...,...,...,...
89,bears-ears,2023-11-04,Edward Del Favero,23393000.0,M,66,None,None
90,bears-ears,2023-11-04,Naik Yusufi,26242000.0,M,33,None,None
91,bears-ears,2023-11-04,Lane Brooks,NaN,M,47,None,None
92,bears-ears,2023-11-04,NancyLynn Evangelisti,NaN,F,53,None,None


In [578]:
abbr_to_state = {
    'AL': 'Alabama',
    'AK': 'Alaska',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming',
    'DC': 'District of Columbia',
    'AS': 'American Samoa',
    'GU': 'Guam',
    'MP': 'Northern Mariana Islands',
    'PR': 'Puerto Rico',
    'VI': 'U.S. Virgin Islands',
    'AA': 'Armed Forces Americas',
    'AE': 'Armed Forces Europe',
    'AP': 'Armed Forces Pacific'
}

downhill_marathon_results_df['gender'] = downhill_marathon_results_df['gender'].replace('F', 'W').replace(['U', 'NB', 'NOT SPECIFIED', '---'], 'X')

downhill_marathon_results_df['state'] = downhill_marathon_results_df['state'].replace({v[:10].upper(): k for k, v in abbr_to_state.items()})
downhill_marathon_results_df['state'] = downhill_marathon_results_df['state'].str.lower()
downhill_marathon_results_df['state'] = downhill_marathon_results_df['state'].replace({v.lower(): k  for k, v in abbr_to_state.items()})
downhill_marathon_results_df['state'] = downhill_marathon_results_df['state'].str.upper()

downhill_marathon_results_df['age'] = pd.to_numeric(downhill_marathon_results_df['age'], errors='coerce')

## Cleaning and Formatting Race Data

In [579]:
fields['"CUT-OFF TIME"*'] = pd.to_timedelta('00:0' + fields['"CUT-OFF TIME"*'])
fields[['QUALIFIERS NOT ACCEPTED', 'FIELD SIZE']] = fields[['QUALIFIERS NOT ACCEPTED', 'FIELD SIZE']].apply(lambda col: col.str.replace(',', '').astype(int))

In [580]:
standards[['MEN', 'WOMEN']] = standards[['MEN', 'WOMEN']].apply(lambda col: pd.to_timedelta(col.str.replace('hrs', 'hr')))

In [581]:
downhill_marathons['Net Downhill'] = downhill_marathons['Net Downhill'].str.replace(',', '').astype(int)
downhill_marathons['Category'] = downhill_marathons['Net Downhill'].apply(lambda x: '3,000-5,999' if 3000 <= x < 6000 else '1,500-2,999')

In [582]:
fields_to_save = fields.copy()
fields_to_save['"CUT-OFF TIME"*'] = fields_to_save['"CUT-OFF TIME"*'].apply(lambda x: x.seconds)
fields_to_save.columns = ['Year', 'Field Size', 'Cutoff Time (s)', 'Qualifiers Not Accepted']
fields_to_save.to_csv('data/fields.csv', index=False)

standards_to_save = standards.copy()
standards_to_save[['MEN', 'WOMEN']] = standards_to_save[['MEN', 'WOMEN']].apply(lambda col: col.astype('str').apply(lambda x: x.split(' ')[2][1:]))
standards_to_save.columns = ['Age Group', 'Men', 'Women']
standards_to_save.to_csv('data/standards.csv', index=False)

downhill_marathons_to_save = downhill_marathons.copy()
downhill_marathons_to_save.sort_values(by='Net Downhill', ascending=False, inplace=True)
downhill_marathons_to_save.columns = ['Name', 'Slug', 'Net Downhill (ft)', 'Year Established', 'Location', 'Category']
downhill_marathons_to_save.to_csv('data/downhill-marathons.csv', index=False)

# Calculating BQ Qualification and Participation Statistics

In [583]:
standards_map = {}
for row in standards.itertuples():
    if row.Index == len(standards) - 1:
        age_max = math.inf
    else:
        age_max = int(typing.cast(str, row._1)[3:5])
    standards_map[age_max] = {
        'M': typing.cast(pd.Timedelta, row.MEN).total_seconds() * 1000,
        'F': typing.cast(pd.Timedelta, row.WOMEN).total_seconds() * 1000
    }

In [584]:
qualifying_windows_map = {
    2023: (pd.Timestamp('2021-09-01'), pd.Timestamp('2022-09-16')),
    2024: (pd.Timestamp('2022-09-01'), pd.Timestamp('2023-09-15')),
    2025: (pd.Timestamp('2023-09-01'), pd.Timestamp('2024-09-13')),
}

In [585]:
divisions_map = {
    39: '18-39',
    44: '40-44',
    49: '45-49',
    54: '50-54',
    59: '55-59',
    64: '60-64',
    69: '65-69',
    74: '70-74',
    79: '75-79',
    math.inf: '80+'   
}

def get_division_for_age(age: int):
    for division in divisions_map:
        if age <= division:
            return divisions_map[division]
    raise ValueError(f'Invalid age: {age}')

In [586]:
boston_marathon_results_df = pd.DataFrame(columns=['gender', 'year', 'place_overall', 'place_gender', 'place_division', 'name', 'team', 'bib', 'half_time', 'finish_time_net', 'finish_time_gun', 'division', 'state_province'])

for year in range(2023, 2025 + 1):
    file_path = f'raw/boston-marathon-{year}.json'
    with open(file_path, 'r') as file:
        data = json.load(file)
    
    for gender in ['M', 'F', 'X']:
        if gender in data:
            results = data[gender]
            temp_df = pd.DataFrame(results)
            
            temp_df['half_time'] = temp_df['half_time'].apply(lambda x: conv_formatted_time_to_millis(x))
            temp_df['finish_time_net'] = temp_df['finish_time_net'].apply(lambda x: conv_formatted_time_to_millis(x))
            temp_df['finish_time_gun'] = temp_df['finish_time_gun'].apply(lambda x: conv_formatted_time_to_millis(x))
            temp_df['year'] = year
            temp_df['gender'] = gender
            temp_df['name'] = temp_df['name'].apply(lambda x: ' '.join(reversed(x.split(', '))))

            boston_marathon_results_df = pd.concat([boston_marathon_results_df, temp_df])
    
boston_marathon_results_df

,gender,year,place_overall,place_gender,place_division,name,team,bib,half_time,finish_time_net,finish_time_gun,division,state_province
0,M,2023,1,1,1,Evans Chebet,–,1,3740000.0,7554000.0,7554000.0,18-39,–
1,M,2023,2,2,2,Gabriel Geay,–,3,3740000.0,7564000.0,7564000.0,18-39,FL
2,M,2023,3,3,3,Benson Kipruto,–,5,3739000.0,7566000.0,7566000.0,18-39,–
3,M,2023,4,4,4,Albert Korir,–,19,3740000.0,7681000.0,7681000.0,18-39,–
4,M,2023,5,5,5,Zouhair Talbi,–,31,3740000.0,7715000.0,7715000.0,18-39,–
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,X,2025,25701,68,–,Neil Mina,–,21592,8348000.0,17559000.0,17955000.0,–,BC
67,X,2025,26282,69,–,Jessie Lowell,–,29359,8459000.0,18190000.0,18731000.0,–,MA
68,X,2025,26998,70,–,Penny Stevenson,–,8280,8930000.0,19118000.0,19709000.0,–,–
69,X,2025,27508,71,–,Christian Rountry,–,31254,9059000.0,19962000.0,20460000.0,–,NY


In [587]:
fields = fields.set_index('YEAR')
downhill_marathons = downhill_marathons.set_index('Slug')

In [588]:
downhill_course_adjustments = {
    1499: 0,
    2999: 5 * 60 * 1000,
    5999: 10 * 60 * 1000,
}

In [589]:
@lru_cache(maxsize=None)
def get_boston_years(date: pd.Timestamp):
    boston_years = []
    for year, (start, end) in qualifying_windows_map.items():
        if start <= date < end:
            boston_years.append(year)
    return boston_years

In [590]:
downhill_marathon_results_df['boston_year'] = downhill_marathon_results_df['date'].apply(get_boston_years)
downhill_marathon_results_df = downhill_marathon_results_df.explode('boston_year')

In [592]:
@lru_cache(maxsize=None)
def get_standard(age: int, gender: str):
    standard = math.inf
    for age_max in standards_map:
        if age <= age_max:
            standard = standards_map[age_max]['M' if gender == 'M' else 'F']
            break
    return standard

@lru_cache(maxsize=None)
def get_cutoff_for_year(year: int):
    return fields.loc[year]['"CUT-OFF TIME"*'].total_seconds() * 1000

def calc_outcomes(row):
    standard = get_standard(row['age'], row['gender'])
    boston_year = row['boston_year']

    is_bq = row['time'] <= standard
    is_under_cutoff = is_bq and row['time'] <= (standard - get_cutoff_for_year(boston_year))

    is_downhill_under_cutoff = is_under_cutoff
    if is_downhill_under_cutoff:
        net_downhill = downhill_marathons.loc[row['slug']]['Net Downhill']
        adjustment = 0
        for threshold in downhill_course_adjustments:
            if net_downhill < threshold:
                break
            adjustment = downhill_course_adjustments[threshold]
        else:
            raise ValueError(f'No downhill adjustment found for net downhill {net_downhill}')
        standard = get_standard(row['age'], row['gender'])
        is_downhill_under_cutoff = row['time'] <= (standard - adjustment - get_cutoff_for_year(boston_year))

    did_run_boston = is_under_cutoff
    if is_under_cutoff:
        mask = (boston_marathon_results_df['year'] == boston_year) & (boston_marathon_results_df['name'] == row['name'])
        basic_mask = mask
        # if row['state']:
        #     mask = mask & (boston_marathon_results_df['state_province'] == row['state'])
        if row['age'] and not pd.isna(row['age']):
            mask = mask & ((boston_marathon_results_df['division'] == '–') | (boston_marathon_results_df['division'] == get_division_for_age(row['age'])) | (boston_marathon_results_df['division'] == get_division_for_age(row['age'] + 5)))
        
        boston_results = boston_marathon_results_df[mask]
        basic_boston_results = boston_marathon_results_df[basic_mask]
        if boston_results.empty:
            if not basic_boston_results.empty:
                print(f'Warning: Found basic match for {row["name"]} in {boston_year}, but no exact match with state or age.')
                print(basic_boston_results.iloc[0])
                print(row)
            did_run_boston = False
    
    return pd.Series({
        'bq': is_bq,
        'under_cutoff': is_under_cutoff,
        'ran_boston': did_run_boston,
        'downhill_under_cutoff': is_downhill_under_cutoff
    })

downhill_marathon_results_df[['bq', 'under_cutoff', 'ran_boston', 'downhill_under_cutoff']] = downhill_marathon_results_df.apply(calc_outcomes, axis=1)

gender                       M
year                      2024
place_overall             4351
place_gender              3880
place_division            2416
name               John Loftus
team                         –
bib                       3570
half_time            5099000.0
finish_time_net     11313000.0
finish_time_gun     11454000.0
division                 18-39
state_province              MA
Name: 3879, dtype: object
slug                                  tucson
date                     2022-12-10 00:00:00
name                             John Loftus
time                                11795000
gender                                     M
age                                     65.0
city                            Laguna Beach
state                                     CA
boston_year                             2024
bq                                      True
under_cutoff                            True
ran_boston                             False
downhill_under_cutoff          

In [595]:
# get counts and proportions
downhill_marathon_results_df\
    .groupby(['slug', 'date', 'boston_year'])[['bq', 'under_cutoff', 'ran_boston', 'downhill_under_cutoff']]\
    .agg(['sum', 'mean'])

bq            \
                                                       sum      mean   
slug                           date       boston_year                  
bears-ears                     2023-11-04 2025          17  0.180851   
cascade-express                2023-09-16 2025          45  0.217391   
                               2024-09-07 2025         138  0.255083   
deseret-news                   2022-07-22 2023          16  0.640000   
                               2023-07-24 2024          49  0.145833   
                               2024-07-24 2025          40  0.101010   
east-canyon                    2021-09-04 2023           6  0.077922   
                               2022-09-03 2023           9  0.134328   
                                          2024           9  0.134328   
                               2023-09-02 2024           8  0.112676   
                                          2025           8  0.112676   
                               2024-08-31 2025          20  0.206186   
hawaii-bird-conservation       2021-12-19 2023          21  0.396226   
                               2022-12-18 2024          35  0.388889   
                               2023-12-17 2025          57  0.537736   
leading-ladies                 2022-08-21 2023         106  0.981481   
                               2023-08-20 2024         177  0.983333   
                               2024-08-18 2025          12  0.230769   
light-at-the-end-of-the-tunnel 2022-06-12 2023         202  0.388462   
                               2023-06-11 2024         297  0.392338   
                               2024-06-09 2025         339  0.397887   
pocatello                      2021-09-04 2023          25  0.221239   
                               2022-09-03 2023          28  0.186667   
                                          2024          28  0.186667   
                               2023-09-02 2024          34  0.189944   
                                          2025          34  0.189944   
                               2024-08-31 2025          39  0.234940   
revel-big-bear                 2021-11-13 2023         478  0.372564   
                               2022-11-12 2024          25  1.000000   
                               2023-11-18 2025         736  0.396125   
revel-big-cottonwood           2022-09-10 2023         271  0.266732   
                                          2024         271  0.266732   
                               2023-09-09 2024         338  0.208642   
                                          2025         338  0.208642   
revel-mt-charleston            2022-04-02 2023         508  0.308815   
                               2023-04-01 2024         555  0.358527   
                               2024-04-06 2025         701  0.401719   
revel-white-mountains          2024-05-05 2025         432  0.255168   
runtastic-nebo                 2022-09-07 2023          29  0.124464   
                                          2024          29  0.124464   
                               2023-08-26 2024          25  0.117925   
                               2024-09-07 2025          62  0.201299   
sun                            2022-01-29 2023          22  0.103286   
                               2023-01-28 2024          19  0.121019   
                               2024-01-27 2025          20  0.129032   
sundance-to-spearfish          2022-09-11 2023          32  0.421053   
                                          2024          32  0.421053   
                               2023-09-10 2024          31  0.360465   
                                          2025          31  0.360465   
                               2024-09-08 2025          28  0.378378   
super                          2022-06-26 2023          49  0.166102   
                               2023-06-17 2024          49  0.161184   
                               2024-06-22 2025          81  0.203518   
tucson                  